In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time
import json


from api_keys import g_key

#  Store Part I results into DataFrame

In [2]:
weather_data = pd.read_csv("../WeatherPy/output_data/cities.csv")
weather_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Hanzhong,33.0728,107.0303,35.04,76,94,1.81,CN,1643928566
1,Boa Vista,2.8197,-60.6733,89.58,43,40,5.75,BR,1643928566
2,Onokhoy,51.9320,108.0747,-20.45,99,54,2.75,RU,1643928567
3,Hilo,19.7297,-155.0900,78.53,68,40,9.22,US,1643928314
4,Wahpeton,46.2652,-96.6059,-1.48,71,0,18.41,US,1643928569
...,...,...,...,...,...,...,...,...,...
551,Birao,10.2849,22.7882,71.96,20,17,5.01,CF,1643928929
552,Aloleng,16.1307,119.7824,78.10,81,19,10.18,PH,1643928930
553,Nakusp,50.2499,-117.8023,26.94,80,100,0.00,CA,1643928930
554,Assiniboia,49.6334,-105.9845,3.02,72,100,19.57,CA,1643928690


#  Humidity Heatmap

In [3]:
gmaps.configure(api_key = g_key)

locations = weather_data[["Lat","Lng"]]

humidity = weather_data["Humidity"]

In [4]:
fig = gmaps.figure(center=(46,-5), zoom_level=2)
max_intensity = np.max(humidity)
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating = False, max_intensity=100, point_radius=3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

#  Create new DataFrame fitting weather criteria

In [5]:
narrow_cities=weather_data.loc[(weather_data["Wind Speed"]<= 10) & (weather_data["Cloudiness"] == 0) & \
                               (weather_data["Max Temp"] >=70) & (weather_data["Max Temp"] <=80)].dropna()

narrow_cities

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
74,Kibala,9.1104,18.3463,76.95,15,0,4.61,TD,1643928617
88,Praia,14.9215,-23.5087,75.74,38,0,6.91,CV,1643928500
127,Sarh,9.1429,18.3923,76.91,15,0,4.61,TD,1643928650
178,São Filipe,14.8961,-24.4956,74.21,72,0,7.61,CV,1643928685
292,Sharjah city,25.3573,55.4033,72.25,60,0,2.30,AE,1643928767
383,Atar,20.5169,-13.0499,71.42,11,0,8.48,MR,1643928836
394,Lázaro Cárdenas,17.9583,-102.2000,79.99,58,0,9.95,MX,1643928840
426,Jesup,31.6074,-81.8854,77.11,64,0,6.91,US,1643928854
461,Nioro,13.3500,-15.7500,77.50,25,0,4.47,GM,1643928867
494,Nouadhibou,20.9310,-17.0347,71.58,53,0,6.91,MR,1643928891


#  Hotel Map


In [6]:
hotel_df = narrow_cities.loc[:,["City", "Country", "Lat", "Lng"]]



hotel_df["Hotel Name"] = ""

hotel_df


,City,Country,Lat,Lng,Hotel Name
74,Kibala,TD,9.1104,18.3463,
88,Praia,CV,14.9215,-23.5087,
127,Sarh,TD,9.1429,18.3923,
178,São Filipe,CV,14.8961,-24.4956,
292,Sharjah city,AE,25.3573,55.4033,
383,Atar,MR,20.5169,-13.0499,
394,Lázaro Cárdenas,MX,17.9583,-102.2000,
426,Jesup,US,31.6074,-81.8854,
461,Nioro,GM,13.3500,-15.7500,
494,Nouadhibou,MR,20.9310,-17.0347,


In [7]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [8]:
for index, row in hotel_df.iterrows():
    lat= row["Lat"]
    lng= row["Lng"]
    city_name = row["City"]
    
    params["location"] = f"{lat},{lng}"
    
    print(f"Gathering Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    results = response['results']
    
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    time.sleep(1)
    
print("----End of Search----")


Gathering Results for Index 74: Kibala.
Closest hotel in Kibala is Prasant Hotels.
------------
Gathering Results for Index 88: Praia.
Closest hotel in Praia is Hotel Oasis Atlantico Praiamar.
------------
Gathering Results for Index 127: Sarh.
Closest hotel in Sarh is Prasant Hotels.
------------
Gathering Results for Index 178: São Filipe.
Closest hotel in São Filipe is Hotel Ocean View & Restaurante Seafood.
------------
Gathering Results for Index 292: Sharjah city.
Closest hotel in Sharjah city is Sheraton Sharjah Beach Resort & Spa.
------------
Gathering Results for Index 383: Atar.
Closest hotel in Atar is Bab Sahara.
------------
Gathering Results for Index 394: Lázaro Cárdenas.
Closest hotel in Lázaro Cárdenas is Baymont by Wyndham Lazaro Cardenas.
------------
Gathering Results for Index 426: Jesup.
Closest hotel in Jesup is Quality Inn.
------------
Gathering Results for Index 461: Nioro.
Missing field/result... skipping.
------------
Gathering Results for Index 494: Nouadh

In [9]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
74,Kibala,TD,9.1104,18.3463,Prasant Hotels
88,Praia,CV,14.9215,-23.5087,Hotel Oasis Atlantico Praiamar
127,Sarh,TD,9.1429,18.3923,Prasant Hotels
178,São Filipe,CV,14.8961,-24.4956,Hotel Ocean View & Restaurante Seafood
292,Sharjah city,AE,25.3573,55.4033,Sheraton Sharjah Beach Resort & Spa
383,Atar,MR,20.5169,-13.0499,Bab Sahara
394,Lázaro Cárdenas,MX,17.9583,-102.2000,Baymont by Wyndham Lazaro Cardenas
426,Jesup,US,31.6074,-81.8854,Quality Inn
461,Nioro,GM,13.3500,-15.7500,
494,Nouadhibou,MR,20.9310,-17.0347,HOtel Tasiast


In [10]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))